## <span style="color:green"><h1><center>DEM Processing and Topographic Wetness Index (TWI)</center></h1></span>
<center>Prepared by <br>
    <b>Jibin Joseph and Venkatesh Merwade</b><br> 
Lyles School of Civil Engineering, Purdue University<br>
joseph57@purdue.edu, vmerwade@purdue.edu<br>
<b><br>
    FAIR Science in Water Resources</b><br></center>


## <span style="color:green">Objective</span>
<p style='text-align: justify;'> The objective of this tutorial is to learn how to merge, project, clip the DEM raster and generate Topographic Wetness Index (TWI) files. We will use the downloaded DEM raster files from USGS National Elevation Dataset from previous DEM Downloading module.</p> 

## <span style="color:green"> Data Source </span>

<p style='text-align: justify;'>USGS DEM Tiles</p>

## <span style="color:green">Overview of steps </span>
<ol type="1">
    <span style="color:red"><li>Read and plot original files</li></span>
    <span style="color:red"><li>Merge or Mosaick the the original files (not needed if only one raster sile is downloaded)</li></span>
     <span style="color:red"><li>Reproject or Transform coordinate system fro Goegraphic to Projected Coordinate System</li></span>
    <span style="color:red"><li>Clipping or Masking the reprojected raster file</li></span>
    <span style="color:red"><li>Generate TWI raster file</li></span>



## <span style="color:green">Import the packages/modules required for this exercise</span>

We need different packages as shown below. It can be either installed using pip method or conda method.


In [ ]:
## Import the modules/packages/libraries required
import math
import matplotlib.pyplot as plt
import os

from pynhd import NLDI
import rasterio
import rasterio.plot

import geopandas as gpd# for reading the shapefile

## Merging/ Mosaicking
from rasterio.merge import merge

## Reprojecting
from rasterio.warp import calculate_default_transform, reproject, Resampling
import pyproj

## Clipping
from rasterio.mask import mask
from shapely.geometry import mapping
from shapely.geometry import Polygon

from os.path import expanduser

import tempfile
from pathlib import Path
import pywbt
import numpy as np

In [ ]:
## Print the version number
import pynhd
print("PyNHD version: ",pynhd.__version__)
del pynhd

print("Rasterio version: ",rasterio.__version__)
print("Geopandas version: ",gpd.__version__)
print("PyProj version: ",pyproj.__version__)

import shapely
print("Shapely version: ",shapely.__version__)
del shapely

print("PyWBT version: ",pywbt.__version__)


## <span style="color:green">Step 1: Define the variables and plot unmerged raster files</span> 
<ul>
<li>Variables: <span style="color:red">USGS Site, resolution, directories, extents</span></li>
<li>Plot Unmerged files: <span style="color:red">using rasterio plot</span></li>

In [ ]:
## Few variables from previous DEM Access Modules
site_id='03363000'
resolution='1'


folder_main=f"{expanduser('~')}/scratch/DEM_Access"
folder_input=f'{folder_main}/data_{site_id}'
dem_files_store=f'{folder_input}/raw_{site_id}'
##Read the shapefile using geopandas
watershed =gpd.read_file(f'{folder_input}/shape_{site_id}.shp')
## Get the extents and number of tiles downloaded
extents_basin=watershed.total_bounds
extent_left=abs(math.floor(extents_basin[0]))
extent_right=abs(math.floor(extents_basin[2]))
extent_bottom=abs(math.ceil(extents_basin[1]))
extent_top=abs(math.ceil(extents_basin[3]))
num_tiles_download=(((extent_left+1)-extent_right)*((extent_top+1)-extent_bottom))
## Define a empty list to hold lon and lat pair
overlap_lonlat=[]

## Create a for loop to create a rectangular boundary and see if overlaps with watershed
for lon in (range(extent_right,extent_left+1,1)):
    for lat in (range(extent_bottom,extent_top+1,1)):
        ## Defining in anticlockwise direction
        corner_left_bottom=(-lon,lat-1)
        corner_right_bottom=(-lon+1,lat-1)
        corner_right_top=(-lon+1,lat)
        corner_left_top=(-lon,lat)
        
        # Create a polygon from the coordinates
        rectangular_boundary = Polygon([corner_left_bottom,corner_right_bottom,corner_right_top,corner_left_top])

        # Create a GeoDataFrame
        rectangular_gdf = gpd.GeoDataFrame(geometry=[rectangular_boundary])
        
        # Set the coordinate reference system (CRS) if needed
        # Example: gdf.crs = {'init': 'epsg:4326'}  # WGS84

        rectangular_gdf.crs = watershed.crs

        # Perform the overlay operation to find the intersection
        intersection = gpd.overlay(watershed, rectangular_gdf, how='intersection')

        # Check if there's any intersection
        if not intersection.empty:
            #print("The rectangular polygon overlaps with the shapefile.")
            overlap_lonlat.append((lon,lat))
        #else:
        #    print("The rectangular polygon does not overlap with the shapefile.")       

print("The required lon and lat pairs are: \n",overlap_lonlat)

## Calulate the number of tiles to be downloaded from USGS
num_tiles_download=(((extent_left+1)-extent_right)*((extent_top+1)-extent_bottom))
print(f"\nNumber of tiles required to cover the entire region: {num_tiles_download}")
print(f"Left: {extent_left}, Right: {extent_right}, Bottom: {extent_bottom}, Top: {extent_top}")

print(f"\nNumber of tiles within watershed boundary: {len(overlap_lonlat)}")

## Creating the dictionary
arc_seconds = {
    "1": "1 arc-second",
    "13": "1/3 arc-second",
    "19": "1/9 arc-second"
}

if (len(overlap_lonlat)>1):
    title=f"Unmerged Raster DEMs\n (for USGS {site_id} and {arc_seconds[resolution]} resolution)"
else:
    title=f"Single Raster DEM\n (for USGS {site_id} and  {arc_seconds[resolution]} resolution)"


## Plotting
fig, ax = plt.subplots(figsize=(8, 8))
for location in overlap_lonlat:
        usgs_filename=f'n{location[1]:02d}w{location[0]:03d}'
        ## WRITE THE CODE BELOW
        local_raster_filename=fr'{dem_files_store}/USGS_{resolution}_{usgs_filename}.tif'
        raster = rasterio.open(local_raster_filename)
        rasterio.plot.show(raster,
                           ax=ax,
                           cmap='viridis')
        #print(f'lat: {lat},lon: {lon},file:{local_fileloc_filename}')
watershed.plot(ax=ax, 
           facecolor='none',
               edgecolor='white',
               linewidth=5)
plt.title(title)
plt.xlabel("Longitude (DD)")
plt.ylabel("Longitude (DD)")

## Free up memory
del raster,fig,ax

## <span style="color:green">Step 2: Merging the raster</span> 

<ul>
<li>Input: <span style="color:red">Downloaded raster tiles</span></li>
<li>Output: <span style="color:red">Merged raster tile</span></li>

## <span style="color:green">Step 2a: Create folders for saving the intermediate and results files</span>
    

In [ ]:
def check_create_path_func(path):
    isExist = os.path.exists(path)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(path)
        print(f"The new directory \033[1m'{path}'\033[0m is created!")
    else:
        print(f"The new directory \033[1m'{path}'\033[0m is not created as it already exists!")
        
folder_process=f"{expanduser('~')}/scratch/DEM_Process"
check_create_path_func(folder_process)

## Create the a folder for intermediate raster files 
## WRITE THE CODE BELOW





## <span style="color:green">Step 2b: Merging the rasters (if required)</span>

<ul>
<li> Define a function for merging the raster files </li>
<li> Create a filename for saving the merged raster files </li>
<li> Call the defined function merging the rasters</li>

In [ ]:
def merge_dem_raster_func(input_foldername, output_filename):
    ## Get a list of all DEM files in the input folder
    dem_files = [f for f in os.listdir(input_foldername) if f.startswith(f"USGS_{resolution}_")]
    ## Create a list to store the raster datasets
    datasets = []
    ## Open each DEM file and append it to the datasets list
    for dem_file in dem_files:
        file_path = os.path.join(input_foldername, dem_file)
        src = rasterio.open(file_path)
        datasets.append(src)
    ## Merge the raster datasets into a single mosaic and return ndarray and information on mapping the pixel coordinates
    ## WRITE THE CODE BELOW
    
    
    
    
    
    ## Copy the metadata from one of the datasets (assuming they all have the same metadata)
    out_meta = datasets[0].meta.copy()

    out_meta.update({
        'height': mosaic_new_raster.shape[1],
        'width': mosaic_new_raster.shape[2],
        'transform': mosaic_transform ## copy the transform parameter from merge
    })
    with rasterio.open(output_filename, 'w', **out_meta) as dest:
        dest.write(mosaic_new_raster)
    ## Close all the opened datasets
    for dataset in datasets:
        dataset.close()
    print(f"Merging completed for {len(overlap_lonlat)} DEM raster files")

if num_tiles_download==1:
    merged_raster_filename=f'{dem_files_store}/USGS_{resolution}_{usgs_filename}.tif'
    print("No need to merge as only one tiff file")
    title_4_plot=f"Original DEM Raster (Merging not required) in {watershed.crs} Geographic CRS\n(for USGS {site_id} and {arc_seconds[resolution]} resolution)"
else:
    merged_raster_filename=f'{folder_intermediate}/merged_{resolution}_{site_id}.tif'
    ## WRITE THE CODE BELOW
    
    title_4_plot=f"Merged DEM Raster in {watershed.crs} Geographic CRS\n(for USGS {site_id} and {arc_seconds[resolution]} resolution)"

In [ ]:
with rasterio.open(merged_raster_filename) as src:
    print(src)
    fig, ax = plt.subplots(figsize = (8,8))
    mapped=rasterio.plot.show(src,
                              ax=ax, 
                              cmap='viridis')
    im = mapped.get_images()[0]
    fig.colorbar(im,
                 ax=ax,
                 shrink=0.5,
                 label='Elevation (in meters)',
                 location='right',
                 orientation='vertical')
    ax.set_xlabel("Longitude (DD)")
    ax.set_ylabel("Latitude (DD)")
    ax.set_title(title_4_plot)
    
watershed.plot(ax=ax,
               facecolor='none',
               edgecolor='white',
               linewidth=5)
src.close()
del mapped,im
del fig,ax

## <span style="color:green">Step 3: Reproject both the shapefile and raster to PCS from GCS</span>

<ul>
<li> Find the UTM Zone for Projected Coordinate System using the centroid point </li>
<li> Project the merged dem </li>

In [ ]:
## Find if basin has a projection
print(watershed.crs)

## <span style="color:green">Step 3a: Finding the UTM Zone</span>

<ul>
<li> Find the approximate centroid </li>    
<li> Find whether the centroid is in N or S </li>
<li> Calculate the zone value (below calculations applciable to conterminous US) </li>


In [ ]:
## Find the approximate centroid
df=watershed
df['geometry'].centroid
## Find the center point
df['Center_point'] = df['geometry'].centroid
## Extract lat and lon from the centerpoint
df["lon"] = df.Center_point.map(lambda p: p.x)
df["lat"] = df.Center_point.map(lambda p: p.y)
df

In [ ]:
## Calculate N or S
if 0 < df['lat'][0] < 84:
    lat_zone='N'
elif -84 < df['lat'][0] < 0:
    lat_zone='S'

## Calculate zone 
## Applicable for US only

if -138 < df['lon'][0] < -132:
    zone_value='08'
elif -132 < df['lon'][0] < -126:
    zone_value='09'
elif -126 < df['lon'][0] < -120:
    zone_value='10'
elif -120 < df['lon'][0] < -114:
    zone_value='11'
elif -114 < df['lon'][0] < -108:
    zone_value='12'
elif -108 < df['lon'][0] < -102:
    zone_value='13'
elif -102 < df['lon'][0] < -96:
    zone_value='14'
elif -96 < df['lon'][0] < -90:
    zone_value='15'
elif -90 < df['lon'][0] < -84:
    zone_value='16'
elif -84 < df['lon'][0] < -78:
    zone_value='17'
elif -78 < df['lon'][0] < -72:
    zone_value='18'
elif -72 and df['lon'][0] < -66:
    zone_value='19'
elif -66 < df['lon'][0] < -60:
    zone_value='20'    
elif -60 < df['lon'][0] < -54:
    zone_value='21'
elif -54 < df['lon'][0] < -48:
    zone_value='22'      
else:
    print("not in conterminous US")
    
crs_string=f'EPSG:326{zone_value}'
crs_string

In [ ]:
## Alternative Method to estimate the CRS
crs_string2=watershed.estimate_utm_crs(datum_name='WGS 84')
crs_string2

## <span style="color:green">Step 3b: Reprojecting watershed shapefile to UTM using calculated zone</span>

<ul>
<li> Using the string created for EPSG (by calculated zone value), reproject it to PCS  </li>
<li> Calculate the zone value (below calculations applciable to conterminous US) </li>

In [ ]:
shapefile_fileloc_filename=f'{folder_input}/shape_{site_id}.shp'
watershed_file=gpd.read_file(shapefile_fileloc_filename)
watershed_proj=watershed_file.to_crs(crs_string)

In [ ]:
print(watershed_file.crs)
print(watershed_proj.crs)

In [ ]:
proj_shapefile_filename=f'{folder_results}/proj_shp_{site_id}.shp'
print("Projected Watershed shapefile is stored at:", proj_shapefile_filename)
watershed_proj.to_file(proj_shapefile_filename, driver='ESRI Shapefile',mode='w')

## <span style="color:green">Step 3c: Reprojecting merged raster to UTM using calculated zone</span>

<ul>
<li> Using the string created for EPSG (by calculated zone value), reproject it to PCS  </li>
<li> Calculate the zone value (below calculations applciable to conterminous US) </li>

In [ ]:
def reproject_raster_func(input_raster, output_raster, target_crs):
    # Load the input DEM
    with rasterio.open(input_raster) as src:
        # Define the target CRS
        target_crs_proj = pyproj.CRS.from_string(target_crs)
        # Calculate the transformation and new dimensions
        transform, width, height = calculate_default_transform(src.crs, target_crs_proj, src.width, src.height, *src.bounds)
        # Update metadata for the new dataset
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs_proj,
            'transform': transform,
            'width': width,
            'height': height
        })
        # Create the output dataset and perform the reprojection
        with rasterio.open(output_raster, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                ## WRITE CODE BELOW
                
                
                
                
                
                
  
        print(f"Reprojection of raster complete to {target_crs} projected CRS.")
reprojected_raster_filename=f'{folder_intermediate}/reprojected_{resolution}_{site_id}.tif'
## WRITE CODE BELOW


In [ ]:
reprojected_raster_filename=f'{folder_intermediate}/reprojected_{resolution}_{site_id}.tif'
with rasterio.open(reprojected_raster_filename) as src:
    fig, ax = plt.subplots(figsize = (8,8))
    mapped=rasterio.plot.show(src,
                              ax=ax, 
                              cmap='viridis')
    im = mapped.get_images()[0]
    fig.colorbar(im,
                 ax=ax,
                 shrink=0.5,
                 label='Elevation (in meters)',
                 location='right',
                 orientation='vertical')
    ax.set_xlabel("Easting (meters)")
    ax.set_ylabel("Northing (meters)")
    ax.set_title(f"Reprojected Raster DEM in {watershed_proj.crs} Projected CRS\n(for USGS {site_id} and {arc_seconds[resolution]} resolution)")
## Uee the reprojected watershed shapefile
watershed_proj.plot(ax=ax,
                    facecolor='none',
                    edgecolor='white',
                    linewidth=5)
src.close()
del mapped,im,fig,ax,src

## <span style="color:green">Step 4: Clipping the reprojected raster file using watershed shapefile</span>

<ul>
<li> Using the buffer, clip/ mask the watershed  </li>


In [ ]:
proj_shapefile_filename=f'{folder_results}/proj_shp_{site_id}.shp'

def clip_raster_with_shapefile_func(input_raster, output_raster, shapefile,buffer_value):
    ## Open the shapefile using geopandas
    shapefile_gdf = gpd.read_file(shapefile).buffer(buffer_value)
    ## Open the input raster
    with rasterio.open(input_raster) as src:
        ## Convert the shapefile geometry to the same CRS as the raster
        shapefile_gdf = shapefile_gdf.to_crs(src.crs)
        ## Convert the shapefile geometry to GeoJSON-like format
        geoms = [mapping(geom) for geom in shapefile_gdf.geometry]
        ## Clip the raster using the shapefile geometry
        ## WRITE THE CODE BELOW
        
        ## Update metadata for the new dataset
        out_meta = src.meta.copy()
        out_meta.update({
            'height': clipped.shape[1],
            'width': clipped.shape[2],
            'transform': out_transform
        })
        ## Write the clipped raster to the output file
        with rasterio.open(output_raster, 'w', **out_meta) as dst:
            dst.write(clipped)
    print(f"Raster clipping complete with a buffer value of {buffer_value} m.")

clipped_raster_filename=f'{folder_results}/clipped_{resolution}_{site_id}.tif'
## WRITE THE CODE BELOW    






In [ ]:
with rasterio.open(clipped_raster_filename) as src1:
    fig, ax = plt.subplots(figsize = (8,8))
    mapped=rasterio.plot.show(src1,
                              ax=ax, 
                              cmap='viridis')
    im = mapped.get_images()[0]
    fig.colorbar(im,
                 ax=ax,
                 shrink=0.5,
                 label='Elevation (in meters)',
                 location='right',
                 orientation='vertical')
    ax.set_xlabel("Easting (meters)")
    ax.set_ylabel("Northing (meters)")
    ax.set_title(f"Clipped Raster DEM with a buffer of {buffer_value_in_meters} m and with {watershed_proj.crs} Projected CRS\n(for USGS {site_id} and {arc_seconds[resolution]} resolution)")

## Uee the reprojected watershed shapefile
watershed_proj.plot(ax=ax,
                    facecolor='none',
                    edgecolor='white',
                    linewidth=5)
del src1,fig,ax

## <span style="color:green">Step 5: Creating TWI raster</span>

<ul>
<li> Intermediate steps: Fill Sinks (Breach Depressions), Flow Direction, Flow Accumulation, Slope and TWI,   </li>


In [ ]:
fname = Path(clipped_raster_filename)
fname.parent.mkdir(parents=True, exist_ok=True)

wbt_args = {
    "BreachDepressions": [f"-i={fname.name}", "--fill_pits", f"-o=dem_corr_{resolution}_{site_id}.tif"],
    "D8Pointer": [f"-i=dem_corr_{resolution}_{site_id}.tif", f"-o=fdir_{resolution}_{site_id}.tif"],
    "DownslopeFlowpathLength": [f"--d8_pntr=fdir_{resolution}_{site_id}.tif", f"-o=downslope_fp_length_{resolution}_{site_id}.tif"],
    "D8FlowAccumulation": [f"-i=fdir_{resolution}_{site_id}.tif", "--pntr", "--out_type='specific contributing area'", f"-o=sca_{resolution}_{site_id}.tif"],
    "Slope": [f"-i=dem_corr_{resolution}_{site_id}.tif", "--units=degrees", f"-o=slope_{resolution}_{site_id}.tif"],
    "WetnessIndex": [f"--sca=sca_{resolution}_{site_id}.tif", f"--slope=slope_{resolution}_{site_id}.tif", f"-o=twi_{resolution}_{site_id}.tif"],
}

In [ ]:
input_dir=folder_results

## WRITE THE CODE BELOW






In [ ]:
with rasterio.open(f"{folder_results}/twi_{resolution}_{site_id}.tif") as src1:
    fig, ax = plt.subplots(figsize = (8,8))
    mapped=rasterio.plot.show(src1,
                              ax=ax, 
                              cmap='plasma')
    im = mapped.get_images()[0]
    fig.colorbar(im,
                 ax=ax,
                 shrink=0.5,
                 label='Topographic Wetness Index\n(Dimensionless)',
                 location='right',
                 orientation='vertical')
    ax.set_xlabel("Easting (meters)")
    ax.set_ylabel("Northing (meters)")
    ax.set_title(f"Topographic Wetness Index (TWI) in {watershed_proj.crs} Projected CRS\n(for USGS {site_id} and {arc_seconds[resolution]} resolution)")

## Uee the reprojected watershed shapefile
watershed_proj.plot(ax=ax,
                    facecolor='none',
                    edgecolor='white',
                    linewidth=5)
del src1,fig,ax

In [ ]:
## Define the file paths for the different rasters

raster_files = {
    "Clipped DEM Raster": [f"{folder_results}/clipped_{resolution}_{site_id}.tif",'Elevation (in meters)'],
    "Breach Depressions Filled Raster": [f"{folder_results}/dem_corr_{resolution}_{site_id}.tif",'Elevation (in meters)'],
    "D8 Flow Direction Raster": [f"{folder_results}/fdir_{resolution}_{site_id}.tif",'Index from 1 to 128'],
    "Slope Raster": [f"{folder_results}/slope_{resolution}_{site_id}.tif",'Slope (Degrees)'],
    "Flow Accumulation Raster (Specific Contributing Area)": [f"{folder_results}/sca_{resolution}_{site_id}.tif",'Count of Cells/ meter'],
    "Topographic Wetness Index (TWI) Raster": [f"{folder_results}/twi_{resolution}_{site_id}.tif",'TWI (Dimensionless)'],}

# Create a figure with a 3x2 layout
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Loop through the raster files and plot them
for ax, (title, (file,units)) in zip(axes, raster_files.items()):
    with rasterio.open(file) as src:
        # Read the first band
        data = src.read(1)
        vmin, vmax = np.percentile(data[data != src.nodata], [2, 98])
        #im = ax.imshow(data, cmap='plasma', vmin=vmin, vmax=vmax)
        
        mapped=rasterio.plot.show(src, ax=ax, cmap='plasma', vmin=vmin, vmax=vmax)
        im = mapped.get_images()[0]
       
    
        # Plot the raster data
        #img = ax.imshow(data, cmap='viridis')
        ax.set_title(title)
        ax.axis('off')  # Hide the axis
        
        ## Uee the reprojected watershed shapefile
        watershed_proj.plot(ax=ax,
                            facecolor='none',
                            edgecolor='white',
                            linewidth=5)
        
        # Add a colorbar specific to this subplot
        cbar = plt.colorbar(im, ax=ax, shrink=0.5)
        cbar.set_label(units)  # Label for the colorbar

# Adjust layout
plt.tight_layout()
plt.show()